##     2.a) 
En Chile, la Norma Técnica de Seguridad y Calidad de Servicio (NTC) establece los límites admisibles de operación para garantizar un servicio eléctrico seguro, confiable y de calidad. A continuación, se detallan los principales límites admisibles de operación según esta normativa:

### 1. Frecuencia

    •Frecuencia nominal: 50 Hz.

    •Desviaciones Permitidas:
        o	En condiciones normales: ±0.2 Hz.
        o   En condiciones de emergencia: ±0.5 Hz.

### 2. Tensión

    •	Tensión Nominal: Dependiendo del nivel de tensión:
        o	Baja Tensión: 220/380 V.
        o	Media y Alta Tensión: Según las especificaciones del sistema.

    •	Variaciones Permitidas:
        o	Baja Tensión: ±10% respecto a la tensión nominal.
        o	Media y Alta Tensión: ±5% respecto a la tensión nominal.

### 3. Cargabilidad de Líneas y Transformadores

    •	Cargabilidad Normal: No debe exceder el 100% de la capacidad nominal del equipo.

    •	Cargabilidad de Emergencia: Puede permitir hasta un 120% de la capacidad nominal por un tiempo limitado, dependiendo de las especificaciones del fabricante y condiciones operativas.

### 4. Factor de Potencia

    •	Límite Inferior: 0.92 inductivo.

    •	Límite Superior: 1.0.

### 5. Interrupciones y Calidad de Servicio

    •	Frecuencia de Interrupciones: Máximo de 6 interrupciones por año por consumidor.

    •	Duración Máxima de Interrupciones: No más de 4 horas por interrupción.

### 6. Distorsión Armónica Total (THD)

    •	Límite para Voltaje: Menor del 5% para niveles de baja tensión.

    •	Límite para Corriente: Según la categoría de la instalación, generalmente menor al 8% para niveles de baja tensión.

### 7. Parpadeo de Voltaje

    •	Límite Admisible:

        o	Pst (Perceived Short-term Flicker Severity): Menor a 1.0.

        o	Plt (Perceived Long-term Flicker Severity): Menor a 0.8.

### 8. Desequilibrio de Tensión

    •	Límite Admisible: El desequilibrio de tensión en sistemas trifásicos no debe exceder el 3%.

### 9. Tiempo de Respuesta ante Contingencias

    •	Recuperación del Servicio: El 90% de los clientes afectados debe tener restablecido el servicio dentro de las 4 horas posteriores a una interrupción.


Estos límites son establecidos para mantener un estándar de calidad y seguridad en el suministro eléctrico, minimizando las interrupciones y asegurando un servicio continuo y de calidad para los usuarios. Es crucial para las empresas eléctricas y operadores del sistema cumplir con estos límites para garantizar la confiabilidad del sistema eléctrico en Chile.

## 2.b) 


### Modelamiento con Parámetros Concentrados

    El modelamiento con parámetros concentrados de una línea de transmisión asume que todos los parámetros eléctricos (resistencia, inductancia, capacitancia y conductancia) se concentran en un punto o en varias secciones discretas a lo largo de la línea. Este modelo es adecuado para líneas de transmisión de longitud corta o media, donde las variaciones a lo largo de la línea no son significativas.

### Modelamiento con Parámetros Distribuidos

    El modelamiento con parámetros distribuidos de una línea de transmisión considera que los parámetros eléctricos están distribuidos de manera continua a lo largo de la línea. Este modelo es más preciso y necesario para líneas de transmisión largas, ya que permite capturar las variaciones de los parámetros eléctricos a lo largo de la línea.

### Diferencias Clave entre los Modelos de Parámetros Concentrados y Distribuidos:

    1.	Precisión:

        o	Parámetros Concentrados: Aproximado y menos preciso, adecuado para líneas cortas y medianas.

        o	Parámetros Distribuidos: Más preciso, esencial para líneas largas.

    2.	Complejidad:

        o	Parámetros Concentrados: Más simple y fácil de implementar.

        o	Parámetros Distribuidos: Más complejo debido a la necesidad de resolver ecuaciones diferenciales.

    3.	Uso en Software:

        o	Parámetros Concentrados: Usado en la mayoría de los software de simulación para simplificar cálculos.

        o	Parámetros Distribuidos: Usado en estudios detallados y específicos donde la precisión es crítica.

### Tipo de Modelamiento en PandaPower

La librería PandaPower trabaja principalmente con el modelamiento de parámetros concentrados. Esto se debe a que este tipo de modelamiento es adecuado para la mayoría de los estudios de red de potencia que requieren simulaciones rápidas y suficientemente precisas para sistemas de longitud corta y media. PandaPower permite la creación de líneas utilizando parámetros concentrados como se muestra en el ejemplo anterior, y no maneja directamente modelos de parámetros distribuidos.

In [1]:
import numpy as np

# Parámetros de la línea
r = 0.02  # Ohm/km
x = 0.115  # Ohm/km
c = 19.1e-9  # F/km
l = 500  # km

# Impedancia serie total de la línea
Z = (r + 1j*x) * l

# Admitancia shunt total de la línea
Y = 1j * 2 * np.pi * 50 * c * l

# Parámetros de la línea
gamma = np.sqrt(Z * Y)
Zc = np.sqrt(Z / Y)

# Longitud total (dos líneas de 500 km cada una)
gamma_total = gamma * 2
Zc_total = Zc

# Matriz ABCD
A = np.cosh(gamma_total)
B = Zc_total * np.sinh(gamma_total)
C = (1 / Zc_total) * np.sinh(gamma_total)
D = A

print("Matriz ABCD:")
print("A =", A)
print("B =", B)
print("C =", C)
print("D =", D)


Matriz ABCD:
A = (0.673804544806203+0.05333534656494724j)
B = (15.630346337017583+102.58288544282654j)
C = (-0.0001119353845693428+0.005333077768303105j)
D = (0.673804544806203+0.05333534656494724j)


In [3]:
import numpy as np

A = (0.673804544806203+0.05333534656494724j)
B = (15.630346337017583+102.58288544282654j)
C = (-0.0001119353845693428+0.005333077768303105j)
D = (0.673804544806203+0.05333534656494724j)
# Datos del receptor
Vr_ref = 500e3 / np.sqrt(3)  # Voltaje fase a neutro (Vr es línea a línea)
S = 1200e6  # Potencia aparente en VA
fp = 0.9

# Potencia activa y reactiva
P = S * fp
Q = S * np.sin(np.arccos(fp))

# Corriente del receptor
Ir = (P - 1j*Q) / (Vr_ref * np.sqrt(3))

# Datos de la fuente
Vs = 500e3 / np.sqrt(3)  # Voltaje fase a neutro

# Calculo del Vr
Vr = (Vs - B * Ir) / A

# Magnitud y ángulo de Vr
Vr_mag = np.abs(Vr)
Vr_angle = np.angle(Vr, deg=True)

print(f"Vr: {Vr_mag:.2f} V, angle: {Vr_angle:.2f} degrees")


Vr: 374000.45 V, angle: -58.80 degrees
